### 数据统计与分析

In [1]:
import re 
import numpy as np
import pandas as pd
import jieba 

from wordcloud import WordCloud

In [62]:
file_path = './data.csv'
data = pd.read_csv(file_path)

> 标题的词云

In [5]:
TITLE_FILE_NAME = './temp/titles.txt'

fp_title = open(TITLE_FILE_NAME, 'w+' , encoding='utf-8')
titles = data['标题']
for title in titles:
  fp_title.write(title[title.find('（'):] if title.find('（') != -1 else ' ')

# 把标题写入文件
fp_title.close()

# 修正标题
data['标题'].apply(lambda x: x[:x.find('（')] if x.find('（') != -1 else x)

0                             人生海海
1                          大话中国艺术史
2                             生死疲劳
3                       蛤蟆先生去看心理医生
4      被讨厌的勇气：“自我启发之父”阿德勒的哲学课 岸见一郎
                  ...             
495                        银河帝国：基地
496                  宫西达也“你肯定能行”绘本
497              天星童书·全球精选绘本：我的小影子
498                    哈利波特20周年纪念版
499                      森林100层的房子
Name: 标题, Length: 500, dtype: object

In [6]:
# 注意到获取的页面内容中有html的标签和其他符号存在，不利于下一步的词云操作，所以需要利用jieba和正则表达式对得到的文本进行清洗
r ='[，。\%、；1234567890n-】【“”]《》（）'

file=open(TITLE_FILE_NAME, "r", encoding='utf-8').read()

# 剔除无关信息
file =re.sub(r,'',file)
# 删除中文无关词汇
chinese_char = '[的版]'
file = re.sub(chinese_char,'',file)

#分词
con = jieba.lcut(file)

#分词后插入空格
words = " ".join(con)

#词云分析
wordcloud = WordCloud(font_path="./assets/1570766042.ttf",background_color="white",width=1300, height=800).generate(words)

wordcloud.to_file('./assets/title_cloud.png') 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.210 seconds.
Prefix dict has been built successfully.


<!-- ![](./assets/title_cloud.png) -->

从词云图中可以看出消费者更加倾向于购买标题中含有“全册”、“推荐”、“代表作”等词的书籍


> 评论数量

In [7]:
# 排序后的DataFrame
data_sort_by_comment = data.sort_values('评论数量', ascending=False)

# 评论数量最多的 top10、30
# data_sort_by_comment.head(10)
# data_sort_by_comment.head(30)

> top10出版社以及上榜的书籍

In [71]:
data.head(5)

,排名,标题,评论数量,推荐率,出版社,作者,出版时间,价格,类别1,类别2
0,1,人生海海（麦家重磅力作，莫言、董卿盛赞，连续两年高居畅销榜，...,894784,100%,北京十月文艺出版社,麦家 著 ，新经典 出品,2019,¥55.00,小说,中国当代小说
1,2,大话中国艺术史（句句有梗，一口气读完10000年中国艺术史。艺术头...,41862,99.90%,海南出版社,意公子；读客文化 出品,2022,¥112.10,艺术,世界各国艺术概况
2,3,生死疲劳（不看不知道，莫言真幽默！全新版本！印章版和普通版随...,209949,100%,浙江文艺出版社,莫言；读客文化 出品,2022,¥66.40,小说,中国当代小说
3,4,蛤蟆先生去看心理医生（年销200万册！英国经典心理咨询入门书，知...,1077140,100%,天津人民出版社,罗伯特·戴博德，果麦文化 出品,2020,¥28.50,心理学,心灵疗愈
4,5,被讨厌的勇气：“自我启发之父”阿德勒的哲学课 岸见一郎,603965,100%,机械工业出版社,"岸見一郎, 古賀史健",2020,¥30.00,成功/励志,心灵与修养


In [8]:
data_groupby_publisher = data.groupby(['出版社'])

# 将 DataFrameGroupBy 转换为 array 类型
# 然后再每一个group（出版社）中添加这个出版社一共出版的图书
# 然后把这个array按照出版图书的降序排序

publisher_with_rank = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_publisher)], dtype=object)])
publisher_with_rank = publisher_with_rank[publisher_with_rank[:,0].argsort()][::-1]

# 所有出版社按照上榜图书排序和他们的图书
# publisher_with_rank

> 一类图书类别统计

In [9]:
data_groupby_type_1 = data.groupby(['类别1'])
type_1_book_group = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_type_1)], dtype=object)])
type_1_book_group = type_1_book_group[type_1_book_group[:,0].argsort()][::-1]

# 所有1级分类和分类下面的图书
# type_1_book_group

> 二类图书类别统计

In [10]:
data_groupby_type_2 = data.groupby(['类别2'])

type_2_book_group = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_type_2)], dtype=object)])
type_2_book_group = type_2_book_group[type_2_book_group[:,0].argsort()][::-1]

# 所有2级分类和分类下面的图书
# type_2_book_group

> 热销图书的定价区间，大家最喜欢什么样的价位的图书

In [57]:
# 价格区间以10为step
data_modify_price = data
data_modify_price = data_modify_price['价格'].apply(lambda x: int(float(x[1:]) / 10))
data_with_price_df = data
# 把价格修正为价格区间
data_with_price_df['价格'] = data_modify_price

In [60]:
# 按照价格分组
data_groupby_price = data_with_price_df.groupby(['价格'])
price_book_group = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_price)], dtype=object)])
price_book_group = price_book_group[price_book_group[:,0].argsort()][::-1]

# 所有的价格区间和区间内的图书数量和内容
# price_book_group

> 出版年份和时间排序，大家最喜欢什么时候出版的书

In [64]:
# 年份以年排序
data_modify_date = data
data_modify_date = data_modify_date['出版时间'].apply(lambda x: x[:4])
data_with_date_df = data
# 把价格修正为价格区间
data_with_date_df['出版时间'] = data_modify_date

In [69]:
# 按照价格分组
data_groupby_date = data_with_date_df.groupby(['出版时间'])
date_book_group = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_date)], dtype=object)])
date_book_group = date_book_group[date_book_group[:,0].argsort()][::-1]

# 所有的价格区间和区间内的图书数量和内容
# date_book_group

### 数据可视化

### 预测分析

In [93]:
from sklearn import ensemble

# 通过学习前450的名称和分类数据, 预测后50个的分类类别.




# 价格修改成整数
# price = [int((float(i[1:]))) for i in data['价格']]

# 训练集和测试集
# x_train, x_test = [[i, i] for i in range(400)], [[i+400, i+400] for i in range(100)]
# y_train, y_test = [[i,price[i]] for i in range(400)], [[i-400,price[i]] for i in range(400, 500)]

# 使用随机森林, 这里使用20个决策树
# model = ensemble.RandomForestRegressor(n_estimators=20)
# model.fit(x_train, y_train)
# score = model.score(x_test, y_test)
# result = model.predict(x_test)


[[0, 55],
 [1, 112],
 [2, 66],
 [3, 28],
 [4, 30],
 [5, 31],
 [6, 39],
 [7, 39],
 [8, 51],
 [9, 34],
 [10, 48],
 [11, 37],
 [12, 58],
 [13, 53],
 [14, 59],
 [15, 171],
 [16, 21],
 [17, 82],
 [18, 59],
 [19, 42],
 [20, 100],
 [21, 39],
 [22, 53],
 [23, 26],
 [24, 34],
 [25, 21],
 [26, 59],
 [27, 139],
 [28, 35],
 [29, 98],
 [30, 20],
 [31, 38],
 [32, 36],
 [33, 45],
 [34, 73],
 [35, 115],
 [36, 45],
 [37, 39],
 [38, 60],
 [39, 37],
 [40, 34],
 [41, 22],
 [42, 49],
 [43, 28],
 [44, 25],
 [45, 20],
 [46, 194],
 [47, 45],
 [48, 30],
 [49, 45],
 [50, 67],
 [51, 49],
 [52, 48],
 [53, 53],
 [54, 135],
 [55, 39],
 [56, 151],
 [57, 53],
 [58, 10],
 [59, 39],
 [60, 32],
 [61, 100],
 [62, 48],
 [63, 28],
 [64, 108],
 [65, 45],
 [66, 39],
 [67, 49],
 [68, 23],
 [69, 74],
 [70, 86],
 [71, 45],
 [72, 49],
 [73, 30],
 [74, 49],
 [75, 41],
 [76, 21],
 [77, 39],
 [78, 59],
 [79, 24],
 [80, 36],
 [81, 22],
 [82, 16],
 [83, 25],
 [84, 39],
 [85, 48],
 [86, 68],
 [87, 45],
 [88, 156],
 [89, 55],
 [90, 156